In [ ]:
import random
from sympy import mod_inverse

class ShamirSecretSharing:
    def __init__(self, prime=97):
        self.prime = prime  # Prime number > secret

    def generate_polynomial(self, secret, k):
        """Generate random polynomial of degree k-1 with the secret as the constant term"""
        coefficients = [secret] + [random.randint(1, self.prime - 1) for _ in range(k - 1)]
        return coefficients

    def evaluate_polynomial(self, coeffs, x):
        """Evaluate polynomial at point x (mod prime)"""
        y = 0
        for i, c in enumerate(coeffs):
            y = (y + c * pow(x, i, self.prime)) % self.prime
        return y

    def generate_shares(self, secret, n, k):
        """Generate n shares using (k, n) threshold scheme"""
        coeffs = self.generate_polynomial(secret, k)
        shares = [(x, self.evaluate_polynomial(coeffs, x)) for x in range(1, n + 1)]
        return shares

    def reconstruct_secret(self, shares, prime):
        """Reconstruct the secret using Lagrange interpolation"""
        secret = 0
        for i, (xi, yi) in enumerate(shares):
            li = 1
            for j, (xj, _) in enumerate(shares):
                if i != j:
                    li *= (xj * mod_inverse(xj - xi, prime)) % prime
                    li %= prime
            secret = (secret + yi * li) % prime
        return secret


# ==== Demonstration ====
if __name__ == "__main__":
    shamir = ShamirSecretSharing(prime=97)
    secret = 42
    n, k = 5, 3

    print(f"Secret: {secret}")
    shares = shamir.generate_shares(secret, n, k)
    print(f"Generated {n} shares:")
    for s in shares:
        print(s)

    # Reconstruct using any k shares
    subset = shares[:k]
    recovered = shamir.reconstruct_secret(subset, shamir.prime)
    print(f"\nReconstructed secret using {k} shares: {recovered}")
    print("Correct reconstruction " if recovered == secret else "Reconstruction failed")
